In [25]:
import networkx as nx
import numpy as np
import random as rd
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})
import sys
from scipy import special
from scipy import interpolate

In [26]:
def fitness_distribution(T, theta, final_nodes): 
    
    """returns an array of fitness generated by the energy distribution g(E) = CE^theta
    using Emax = 1"""
    
    C = theta + 1 #normalization constant

    energies = (np.random.rand(final_nodes) * (theta + 1)/ C) ** (1/(theta + 1)) 
    
    fitness = np.exp(-energies / T) 
    
    
    return fitness #c = 2


#print(dist)

def sum_fitness(G, fitness_dist):
    """partion function"""
    degrees = np.array([val for (node, val) in G.degree()])
    dist_existingnodes = fitness_dist[:G.number_of_nodes()]
    denominador = np.dot(degrees, dist_existingnodes)
    return denominador


def rand_prob_node(G,dist, alpha):
    """chooses which existing node is going to connected with the new added node, 
    using a fitness model"""
    nodes_probs = []
    
    degrees = np.array([val for (node, val) in G.degree()])
    dist_existingnodes = dist[:G.number_of_nodes()]
    denominador = np.dot(degrees**alpha, dist_existingnodes)
    
    for node in G.nodes():
        node_degr = G.degree(node)

        node_proba = node_degr**alpha * dist[node] / (denominador)
        

        nodes_probs.append(node_proba)

    nodes_probs /= sum(nodes_probs)
    random_proba_node = np.random.choice(G.nodes(),p=nodes_probs)
    
    return random_proba_node


def add_edge(G,dist,new_node, alpha):

    if len(G.edges()) == 0:
            random_proba_node = 0
    else:
        random_proba_node = rand_prob_node(G,dist, alpha)
    new_edge = (random_proba_node, new_node)
    if new_edge in G.edges():
        add_edge(G,dist,new_node)
    else:
        G.add_edge(new_node, random_proba_node)
        #print("Edge added: {} {}".format(new_node + 1, random_proba_node))

In [27]:
def simul(T, m, init_nodes, final_nodes, alpha, theta): 
    """simulates a network with the distribution given by fitness_distribution for
    a given temperature 'T', 'm' and 'initial_nodes', 'final_nodes', 'alpha'
    and 'theta'. Returns chemical potential and kmax/mt"""
    
    G = nx.complete_graph(init_nodes)
    
    count = 0
    new_node = init_nodes
    dist = fitness_distribution(T, theta, final_nodes)


    for f in range(final_nodes - init_nodes):

        G.add_node(init_nodes + count)
        #print(len(G.nodes))
        count += 1
        for e in range(m):
                add_edge(G,dist,new_node, alpha)
        new_node += 1

    fitness_sum = sum_fitness(G, dist)
    chemical_potential = np.log(m*(final_nodes - init_nodes) /fitness_sum)*T
    #print(chemical_potential)
    #print("\nFinal number of nodes ({}) reached".format(len(G.nodes())))
    max_degree = np.max(np.array(list(G.degree()))[:,1])

    return chemical_potential, max_degree/(m * (final_nodes - init_nodes))
